**Install TensorFlow and other dependencies**

In [ ]:
!pip install --upgrade pip
!pip install --upgrade protobuf 

In [ ]:
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

**Check GPU status**

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

**Mount Google Drive folder**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
#%cd '/content/gdrive/My Drive/models/'

In [ ]:
# I change My\ Drive/ to mydrive for easier

!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# I am controlling the contents of models folder

!ls /content/gdrive/MyDrive/models

**Install protobuf and compile, install setup.py**

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/gdrive/MyDrive/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/models/research/:/content/gdrive/MyDrive/models/research/slim'

!python /content/gdrive/MyDrive/models/research/slim/setup.py build
!python /content/gdrive/MyDrive/models/research/slim/setup.py install

**Check remaining GPU time**

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.735525415605968


# ***Start training***

In [ ]:
!pip install tf_slim
%cd /content/gdrive/MyDrive/models/research/object_detection
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/models/research/:/content/gdrive/MyDrive/models/research/slim'

!python train.py --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config --logtostderr

***Export inference graph***

In [ ]:
#  .ckpt needs to be updated every time to match last .ckpt generated
#  .config needs to be updated when changing model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-6325 --output_directory new_graph

Zip file in Google Drive

In [ ]:
!zip -r model_graph.zip new_graph

## **Test**


*   **40 images tested and detected.**
*   **Some videos tested**



***Run the code "custom_model_images" to test the images.***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/custom_model_images.py

***If it is not working, run the below code***

In [ ]:
# !pip install lvis

***Run the code "custom_model_video" to test the video.***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/custom_model_video.py

***Summary metrics of model***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/model_main.py --pipeline_config_path=/content/gdrive/MyDrive/models/research/object_detection/training/ssd_mobilenet_v1_pets.config --model_dir=/content/gdrive/MyDrive/models/research/object_detection/new_graph --eval_training_data=True --checkpoint_dir=/content/gdrive/MyDrive/models/research/object_detection/training --run_once=True


***Confusion matrix in csv format***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/confusion_matrix.py --detections_record=/content/gdrive/MyDrive/models/research/object_detection/data/out.record --label_map=/content/gdrive/MyDrive/models/research/object_detection/training/object-detection.pbtxt --output_path=/content/gdrive/MyDrive/models/research/object_detection/data/confusion_matrix.csv
